In [0]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import initializers as init

# from tensorflow.keras.backend import set_session
import keras.backend as K
import nltk
import string
import numpy as np
import pandas as pd
import re
from tqdm import tqdm


In [0]:
def binary(y):
  if y>=0.5:
      return 1
  else:
      return 0
def preprocess(text):
  text = re.sub(r'\W+',' ',text.lower())
  return text
  

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
datapath = 'gdrive/My Drive/data/'

In [0]:
df = pd.read_csv(datapath+'ml/train.csv')



In [11]:
df.shape

(1804874, 45)

In [0]:
x = df['comment_text'].apply(preprocess)
y = df['target'].apply(binary)

In [13]:
print(x.head())
print(y.head())
print(len(x),len(y))

0    this is so cool it s like would you want your ...
1    thank you this would make my life a lot less a...
2    this is such an urgent design problem kudos to...
3    is this something i ll be able to install on m...
4                 haha you guys are a bunch of losers 
Name: comment_text, dtype: object
0    0
1    0
2    0
3    0
4    1
Name: target, dtype: int64
1804874 1804874


In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

In [0]:
vocabulary_size = len(tokenizer.word_index)+1


In [29]:
print(vocabulary_size)

309012


In [17]:
MAX_LEN=150
sequences = tokenizer.texts_to_sequences(x)
tr_data = pad_sequences(sequences, maxlen=MAX_LEN)

print(len(tr_data))


1804874


In [18]:
f = open(datapath+'glove.6B.300d.txt')
embedding_values = {}
for line in tqdm(f):
  value = line.split(' ')
  word = value[0]
  coef = np.array(value[1:],dtype = 'float32')
  embedding_values[word]=coef

400000it [00:34, 11596.67it/s]


In [19]:
embedding_matrix = np.zeros((vocabulary_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
  values = embedding_values.get(word)
  if values is not None:
    embedding_matrix[i] = values

100%|██████████| 309011/309011 [00:00<00:00, 604022.48it/s]


In [0]:
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# set_session(tf.Session(config=config))

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr_data,y,test_size = 0.25,random_state = 42)

In [0]:
# del x,y,tr_data

In [0]:
import numpy as np
import tensorflow as tf
import random as rn
import os
from numpy.random import seed
from tensorflow import set_random_seed


def session_refresh():
  seed(2)
  set_random_seed(2)
  rn.seed(2)
  os.environ['PYTHONHASHSEED'] = '0'
  session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
  K.set_session(sess)
  print('refreshed session')

In [0]:
embeddingInit = init.RandomUniform(seed=2)
kernelInit = init.glorot_uniform(seed=2)
recurrentInit = init.Orthogonal(seed=2)

In [36]:


session_refresh()
# K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size,300, input_length = MAX_LEN,weights = [embedding_matrix],trainable = False, embeddings_initializer=embeddingInit))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
# model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True),merge_mode='ave'))
# model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2),merge_mode='ave'))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True,kernel_initializer=kernelInit,recurrent_initializer=recurrentInit)))#,merge_mode='ave'
model.add(Dropout(0.2,seed=2))
model.add(Bidirectional(CuDNNLSTM(100,kernel_initializer=kernelInit,recurrent_initializer=recurrentInit)))
model.add(Dropout(0.2,seed=2))
model.add(Dense(64,activation = 'relu',kernel_initializer=kernelInit))
model.add(Dense(1, activation='sigmoid',kernel_initializer=kernelInit))
adg = optimizers.Adagrad(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#'adam'

filepath=datapath+'ml/models/'+"tox-ada-relu-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()
# model.fit(tr_data, y, validation_split=0.25, epochs=3,batch_size=128)
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=4,callbacks=callbacks_list,batch_size=500)

refreshed session
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 300)          92703600  
_________________________________________________________________
bidirectional_7 (Bidirection (None, 150, 200)          321600    
_________________________________________________________________
dropout_7 (Dropout)          (None, 150, 200)          0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 200)               241600    
_________________________________________________________________
dropout_8 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                12864     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65 

In [37]:
session_refresh()
# K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size,300, input_length = MAX_LEN,weights = [embedding_matrix],trainable = False, embeddings_initializer=embeddingInit))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
# model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True),merge_mode='ave'))
# model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2),merge_mode='ave'))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True,kernel_initializer=kernelInit,recurrent_initializer=recurrentInit)))#,merge_mode='ave'
model.add(Dropout(0.2,seed=2))
model.add(Bidirectional(CuDNNLSTM(100,kernel_initializer=kernelInit,recurrent_initializer=recurrentInit)))
model.add(Dropout(0.2,seed=2))
model.add(Dense(64,activation = 'relu',kernel_initializer=kernelInit))
model.add(Dense(1, activation='sigmoid',kernel_initializer=kernelInit))
adg = optimizers.Adagrad(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])#'adam'

filepath=datapath+'ml/models/'+"tox-ada-relu-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()
# model.fit(tr_data, y, validation_split=0.25, epochs=3,batch_size=128)
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=4,callbacks=callbacks_list,batch_size=500)

refreshed session
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 150, 300)          92703600  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 150, 200)          321600    
_________________________________________________________________
dropout_9 (Dropout)          (None, 150, 200)          0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 200)               241600    
_________________________________________________________________
dropout_10 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                12864     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65 

In [0]:
model_json = model.to_json()
with open(datapath+"ml/models/tox-ada-relu-model.json", "w") as json_file:
    json_file.write(model_json)

In [0]:
newtest= pd.read_csv(datapath+'ml/test.csv')
xt = newtest['comment_text'].apply(preprocess)
sequences = tokenizer.texts_to_sequences(xt)
ts_data = pad_sequences(sequences, maxlen=MAX_LEN)



In [0]:
yts= model.predict(ts_data)

In [0]:
netdata = pd.DataFrame([newtest['id']]).T


In [0]:
netdata['prediction'] = yts

In [49]:
newtest['prediction'] = yts
newtest.head()

,id,comment_text,prediction
0,7000000,Jeff Sessions is another one of Trump's Orwell...,0.003280
1,7000001,I actually inspected the infrastructure on Gra...,0.000100
2,7000002,No it won't . That's just wishful thinking on ...,0.003977
3,7000003,Instead of wringing our hands and nibbling the...,0.001472
4,7000004,how many of you commenters have garbage piled ...,0.987436


In [0]:
newtest.to_csv('newdata.csv', index=False)